In [1]:
import pandas as pd
import numpy as np
import os
import re
import requests
import geopandas
import geopy
import requests, json
import urllib.parse
from selenium import webdriver
from tqdm import tqdm_notebook as tqdmn
import folium

In [3]:
df = pd.read_excel("Liste_theatre_corpus_1_2 copie.xlsx")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [4]:
df_final_corpus_1 = df
df_final_corpus_1.head(3)

,Theatre
0,Abbey Road Studios
1,Abbey Theatre
2,Abney Park Cemetery


In [5]:
list_theatre_update = []

for i in range(0,len(df_final_corpus_1["Theatre"])):
    if "theatre" not in str(df_final_corpus_1["Theatre"][i]).lower():
        list_theatre_update.append(str(df_final_corpus_1["Theatre"][i]) + " Theatre" + " London UK")
    else:
        list_theatre_update.append(str(df_final_corpus_1["Theatre"][i]) + " London UK")
        

df_final_corpus_1['Theatre_New'] = list_theatre_update
df_final_corpus_1.head(3)

,Theatre,Theatre_New
0,Abbey Road Studios,Abbey Road Studios Theatre London UK
1,Abbey Theatre,Abbey Theatre London UK
2,Abney Park Cemetery,Abney Park Cemetery Theatre London UK


In [6]:
df_final_corpus_1['Url'] = ['https://www.google.com/maps/search/' + i for i in df_final_corpus_1['Theatre_New']]

In [8]:
df_final_corpus_1.to_excel("theatre.xlsx",index=False)

In [7]:
Url_With_Coordinates = []
option = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
option.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome("/Users/gaetanbrison/Desktop/chromedriver", options=option)
for url in tqdmn(df_final_corpus_1.Url, leave=False):
    driver.get(url)
    Url_With_Coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))
driver.close()

/Users/gaetanbrison/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/496 [00:00<?, ?it/s]

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"meta[itemprop=image]"}
  (Session info: chrome=96.0.4664.110)


In [ ]:
df_final_corpus_1['Url2'] = ""

In [ ]:
df_final_corpus_1['Url2'] = Url_With_Coordinates

In [ ]:
list_lat = []
list_long = []
for url in df_final_corpus_1['Url2']:
    if ("?center=" in url) & ("&zoom=" in url) & ("%2C" in url):
        list_lat.append(url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0])
        list_long.append(url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1])
    else:
        list_lat.append(0.0)
        list_long.append(0.0)

In [ ]:
df_final_corpus_1['Lat'] = list_lat
df_final_corpus_1['Long'] = list_long